# HW 4：文字雲 - 兩黨比較

比較兩黨近期臉書貼文內容。

--------------------------------------------------------------------------------------------------------------------------------
## Import Data

In [11]:
import requests
import pandas as pd
from dateutil.parser import parse

In [55]:
#from wordcloud import WordCloud
import matplotlib.pyplot as plt
import jieba
% matplotlib inline

In [24]:
token = 'EAACEdEose0cBAHFIehnM3GRlGqNHxQxR5k8wYEJbHA1866qNdDLfkrTZBjnN3Ym5XZAFUvQcDZCdpiZCpElK9JBUkgwTM2awemLVPPsCV4s7xvDZAfZC5bmPFGHhZBZAJZCwd9A1vZArrRyZAFbZBO628fR2CLPewBbQsIAdLctfNNq30ZAnfK5nAEsUxpXHuZBVR7uIko6DZBQW8olruy5HW6EO58ZA' 

fb_id_1 = "150336441057"
fb_id_2 = "131181017972"

## Post Scraping

In [41]:
def fbcrawler(fb_id):
    columns = ['發文時間', '文章內容', '分享內容', '留言數', '按讚數', '分享數']
    posts = []
    res = requests.get('https://graph.facebook.com/v2.9/{}/posts?limit=20&access_token={}'.format(fb_id, token))
    for post in res.json()['data']:
        p_res = requests.get(
                'https://graph.facebook.com/v2.9/{}?'.format(post['id']) +
                'fields=comments.limit(0).summary(true),likes.limit(0).summary(true), shares&' +
                'access_token={}'.format(token))
        # 留言數
        if 'comments' in p_res.json():
            comments = p_res.json()['comments']['summary'].get('total_count')
        else:
            comments = 0
        # 按讚數
        if 'likes' in p_res.json():
            likes = p_res.json()['likes']['summary'].get('total_count')
        else:
            likes = 0
        # 分享數
        if 'shares' in p_res.json():
            shares = p_res.json()['shares'].get('count')
        else:
            shares = 0
        # 將資訊保存
        posts.append([parse(post['created_time']).date(),
                      post.get('message'),
                      post.get('story'),
                      comments,
                      likes,
                      shares])
    # 使用 pandas 將資料輸出
    df = pd.DataFrame(posts, columns=columns)
    return df

In [46]:
# 民進黨
df1 = fbcrawler(fb_id_1)
df1.head(10)

,發文時間,文章內容,分享內容,留言數,按讚數,分享數
0,2018-03-28,時間，是我們共同的對手。\n \n為使民眾能更加了解「長照 2.0」政策與各項涵蓋服務內容，...,None,17,92,3
1,2018-03-28,常會後記者會，轉述國發會政務次長邱俊榮「今年經濟情勢發展」會中報告。\n\n由發言人吳思瑤主持,民主進步黨 was live.,100,469,24
2,2018-03-26,【民進黨推出《人生一瞬》桌遊 透過遊戲宣傳長照政策】\n \n民主進步黨今(26)日由黨副秘...,民主進步黨 was live.,127,442,12
3,2018-03-23,【在地生活新趣味│新竹市】\n風起時聽歌、搖擺 #風搖滾 開幕嘉年華正式開唱\n\n新竹市第...,民主進步黨 shared 新竹市 新鮮事's post.,49,97,2
4,2018-03-21,【民主進步黨第十七屆第六十八次中常會新聞稿】\n \n民主進步黨今(21)日召開第十七屆第六...,民主進步黨 was live.,148,363,11
5,2018-03-21,【洪耀福: 放下歧見、共同團結，讓嘉義縣更發展】\n \n民進黨秘書長洪耀福今(21)日偕同...,民主進步黨 was live.,128,1089,62
6,2018-03-18,【鄭朝方、徐國勇攜手合作 客家便當竹東飄香】\n \n民進黨客家部於今（18）日於新竹縣舉辦...,民主進步黨 was live.,175,504,12
7,2018-03-16,【在地生活新趣味│高雄市】\n \n2018 台灣國際遊艇展 港都盛大登場\n \n你知道嗎...,民主進步黨 shared Taiwan Int'l Boat Show 台灣國際遊艇展's ...,67,242,10
8,2018-03-16,【針對今日《信傳媒》報導之澄清與說明】\n \n針對今(16)日信傳媒稍早題為「游錫堃被拱加...,None,74,516,3
9,2018-03-16,【原民心動力．原民好選擇 2018直轄市原住民議員首波徵召名單】\n \n民進黨3/14率先...,None,92,661,20


In [45]:
# 國民黨
df2 = fbcrawler(fb_id_2)
df2.head(10)

,發文時間,文章內容,分享內容,留言數,按讚數,分享數
0,2018-03-28,【對的人】\n愛要耐心等待、仔細尋找，「對象」很重要！\n找個高薪的人來湊，平均月薪200K...,None,34,194,15
1,2018-03-27,乾淨的「沒」\n—民進黨的能源政策。\n#缺電危機\n#空污治國\n#用肺發電,None,37,304,39
2,2018-03-23,【台灣漁民就是比較活潑啦】\n駐日代表謝長廷認為我國漁民比較「活潑」，哪裡都去；日本漁民就在...,None,75,270,24
3,2018-03-22,在臨近329青年節的今日，我們以看電影、做公益的方式，緬懷革命先烈紀念日\n————————...,None,148,508,59
4,2018-03-22,【國人健康也得分藍綠？】\n國民黨執政時期，時任行政院長張善政動用第二預備金添購公費流感疫苗...,None,25,258,40
5,2018-03-22,Tell Me 餵什麼\n知福、惜福、再造福\n愛福、享福、再施福，\n大來賓：許淑華 委員...,中國國民黨 KMT was live.,191,937,55
6,2018-03-20,【蔡英文髮夾彎日常，你抓緊了嗎】\n\n從「不缺電」到「沒有深澳發電廠備轉容量將不足」、從「...,None,107,414,191
7,2018-03-19,【荒唐政府的荒謬政策】\n只會選舉不會執政的政黨，為了選票不惜欺騙人民，缺乏長遠規劃的政見永...,None,50,635,190
8,2018-03-15,【能源政策與環境永續】\n空污對人體造成的危害超乎您的想像，未來供電吃緊、發電廠火力全開，人...,None,81,399,73
9,2018-03-15,Tell Me 餵什麼\n貞心守護 國民至上\n大來賓：江惠貞 新北果菜公司總經理\n主持人...,中國國民黨 KMT was live.,92,638,43


## Text Mining

### Step 1: Create a Word Dictionary

In [52]:
def WordDict(df):
    post = df['文章內容'].astype(str)
    post = ''.join(post)
    
    stopwords = ['我', '對', '的','和']
    puncs = '！？｡＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏.。'
    
    seg_list = jieba.cut(post, cut_all=False)
    seg_list = list(seg_list)
    wordDict = {}
    for w in seg_list:
        if (w not in stopwords) and (w not in puncs):
            if w in wordDict:
                wordDict[w] = wordDict[w] + 1
            else:
                wordDict[w] = 1
    print(wordDict)
    return

In [50]:
wordDict1 = WordDict(df1)

{'時間': 5, '是': 14, '我們': 11, '共同': 5, '手': 1, '\n': 164, '\u3000': 65, '為': 22, '使民眾': 1, '能': 4, '更加': 2, '了解': 4, '長': 35, '照': 16, ' ': 139, '2.0': 3, '政策': 14, '與': 18, '各項': 2, '涵蓋': 1, '服務': 5, '內容': 3, '本黨': 5, '台灣': 9, '本土': 1, '文創': 1, '團隊': 3, '跨界': 1, '合作': 3, '由': 5, '深受': 1, '好評': 1, '桌遊': 7, '創意': 2, '擔任': 1, '顧問': 1, '開發': 1, '《': 8, '人生': 7, '一瞬': 7, '在': 37, '模擬長': 1, '照現': 1, '實環境': 1, '傳': 1, '逹': 1, '正': 1, '確': 1, '知': 1, '識': 1, '前提': 2, '下': 4, '使': 1, '玩家': 1, '通力合作': 1, '遊戲': 6, '過程': 5, '中': 11, '瞭解長': 1, '推行': 2, '現況': 1, '並': 2, '一同': 3, '為長': 1, '照環境': 1, '努力': 6, '桌遊將': 1, '於': 17, '四月': 1, '首波': 7, '公開': 1, '發行': 1, '追': 1, '蹤': 1, '更': 15, '多': 11, '最新消息': 1, 'https': 17, ':': 20, '/': 60, 'goo': 17, 'gl': 17, 'forms': 2, 'kl0n6Rw4esSQ9WiI2': 2, '_': 1, '洪添賢': 3, 'Croter': 5, 'Hung': 2, 'Illustration': 2, '&': 2, 'Design': 2, 'Studio': 4, '迷走': 3, '工作坊': 3, 'Teenage': 2, 'Riot': 2, '狼': 2, '圖騰': 2, '工作室': 2, 'Wolf': 2, 'Totem': 2, 'Jesse': 2, 'Li': 2, '常

In [51]:
wordDict2 = WordDict(df2)

{'人': 7, '\n': 96, '愛要': 1, '耐心': 1, '等待': 1, '仔細': 1, '尋找': 1, '對象': 1, '很': 1, '重要': 1, '找個': 1, '高薪': 1, '人來': 2, '湊': 1, '平均': 2, '月薪': 2, '200K': 1, '不是': 2, '夢': 1, '#': 27, '勞動': 1, '部長': 1, '妳': 1, '確定': 1, '有': 3, '找': 1, '統計': 1, '嗎': 3, '來': 2, '算算': 1, '您': 3, '多少': 1, '那': 1, '消失': 1, '50K': 1, '薪水': 1, '小偷': 1, '乾淨': 2, '沒': 1, '民進': 3, '黨': 3, '能源': 3, '政策': 3, '缺電': 2, '危機': 1, '空污治國': 1, '用': 2, '肺': 2, '發電': 2, '台灣': 5, '漁民': 2, '就是': 1, '比較': 1, '活潑': 2, '啦': 1, '駐日': 1, '代表': 3, '謝長': 1, '廷': 1, '認為': 1, '我國': 1, '漁民比': 1, '較': 1, '哪裡': 1, '都': 3, '去': 1, '日本': 1, '就': 5, '在': 10, '自己': 3, '海域': 1, '內作業': 1, '做事': 1, '嚴謹': 1, '這種': 2, '說法': 2, '豈': 1, '漁業': 1, '談判': 1, '前': 1, '矮人一截': 1, '中華民國': 3, '駐': 1, '日': 6, '維': 1, '繫': 1, '台': 1, '情': 1, '誼': 1, '時': 3, '同時': 1, '也': 4, '得': 1, '捍衛': 5, '漁民權益': 2, '與': 7, '國家': 2, '尊嚴': 1, '否則': 1, '誰': 1, '臨近': 1, '329': 1, '青年': 4, '節的': 1, '今日': 1, '我們': 5, '以': 2, '看': 1, '電影': 3, '做': 4, '公益': 4, '方式': 1, '緬懷': 2, '革命先烈

### Step 2: Create Word Cloud

In [53]:
font = r"/usr/share/fonts/truetype/wqy/wqy-microhei.ttc"
wordcloud = WordCloud(font_path=font)

NameError: name 'WordCloud' is not defined

In [ ]:
f = plt.figure()

# 民進黨
f.add_subplot(1,2, 1)
wordcloud.fit_words(wordDict1)
plt.imshow(wordcloud)

# 國民黨
f.add_subplot(1,2, 2)
wordcloud.fit_words(wordDict2)
plt.imshow(wordcloud)

plt.show(block=True)

In [ ]:
columns = ['發文時間', '文章內容', '分享內容', '留言數', '按讚數', '分享數']
# 用來保存每篇文章的資料
posts = []
# 將粉絲專頁的 id 及 token 帶入並取得 response
res = requests.get('https://graph.facebook.com/v2.9/{}/posts?limit=20&access_token={}'.format(fb_id_1, token))
for post in res.json()['data']:
    # 透過文章的id來取得 留言數、按讚數、分享數
    p_res = requests.get(
        'https://graph.facebook.com/v2.9/{}?'.format(post['id']) +
        'fields=comments.limit(0).summary(true),likes.limit(0).summary(true), shares&' +
        'access_token={}'.format(token))
    # 留言數
    if 'comments' in p_res.json():
        comments = p_res.json()['comments']['summary'].get('total_count')
    else:
        comments = 0
    # 按讚數
    if 'likes' in p_res.json():
        likes = p_res.json()['likes']['summary'].get('total_count')
    else:
        likes = 0
    # 分享數
    if 'shares' in p_res.json():
        shares = p_res.json()['shares'].get('count')
    else:
        shares = 0
    # 將資訊保存
    posts.append([parse(post['created_time']).date(),
                  post.get('message'),
                  post.get('story'),
                  comments,
                  likes,
                  shares])
# 使用 pandas 將資料輸出
df1 = pd.DataFrame(posts, columns=columns)